In [1]:
import os
import torch
from utils.io import load_yaml
from lightning_modules import DeepImageHomographyEstimationModuleBackbone

# load best model
prefix = '/home/martin/Tresors/homography_imitation_learning_logs/deep_image_homography_estimation_backbone/version_0'
configs = load_yaml(os.path.join(prefix, 'configs.yml'))
model = DeepImageHomographyEstimationModuleBackbone.load_from_checkpoint(os.path.join(prefix, 'checkpoints/epoch=19.ckpt'), shape=configs['model']['shape'])

device = 'cpu'
if torch.cuda.is_available():
    print('Running with CUDA backend.')
    device = 'cuda'

model.to(device)
model = model.eval()

In [11]:
# create dataset and evaluate model on
#   - with motion dataset
#   - robust mis
#   - cholec80
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from kornia import warp_perspective, tensor_to_image
from torchvision.transforms import ToTensor
from datasets import ImageSequenceDataset
from utils.processing import image_edges, four_pt_to_matrix_homography_representation

# 
prefix = '/media/martin/Samsung_T5/data/endoscopic_data/camera_motion_separated_png/with_camera_motion'
df_name = 'log_with_camera_motion_seq_len_20.pkl'
df = pd.read_pickle(os.path.join(prefix, df_name))
ds = ImageSequenceDataset(df, prefix, ToTensor())

for img_seq in ds:
    duv = model(img_seq[0].unsqueeze(0), img_seq[1].unsqueeze(0))

    uv = image_edges(img_seq[0].unsqueeze(0))
    H = four_pt_to_matrix_homography_representation(uv, duv)

    wrp = warp_perspective(img_seq[0].unsqueeze(0), torch.inverse(H), img_seq[0].shape[-2:])
    
    wrp = tensor_to_image(wrp)
    img = tensor_to_image(img_seq[1])

    cv2.imshow('img', img)
    cv2.imshow('wrp', wrp)
    cv2.imshow('L1', abs(img-wrp))
    cv2.waitKey()